In [37]:
# Implements LogRISE algorithm for the inverse Potts problem
# Version: L1 LBFGS with Smoothing
# Description: Uses L1 regularization with logRISE 

import numpy as np
import pandas as pd
from scipy.optimize import minimize,LinearConstraint, fmin_l_bfgs_b
import time
import matplotlib.pyplot as plt
from numba import jit


# Read arguments from file
args = pd.read_csv("arguments.csv",header=None)

# Define arguments
lamda_field = args.iloc[0,0]
lamda_edge = args.iloc[0,1]
file_samples_histo = args.iloc[0,2].strip()
file_reconstruction = args.iloc[0,3].strip()

# Redefining L2 regularization value here. Delete when all arguments specified in argument file. 
lamda_field = 5.0
lamda_edge = 0.1

# Create histogram of configurations
samples_histo = pd.read_csv(file_samples_histo,header=None).values

num_conf,num_row = samples_histo.shape # number of unique samples, length of chain+1
num_spins = num_row-1 # length of chain
num_samples = samples_histo[:,0].sum() #frequency of each sample
num_classes = len(np.unique(samples_histo[:,1:])) # number of states
num_params = num_spins*num_classes**2 # number of parameters

# Array to store parameters
reconstruction = np.empty((num_spins,num_spins,num_classes,num_classes))

# Functions to convert between flattened and n-dimensional parameter array
def to1D(x): 
    return x.flatten()

def toND(vec): 
    assert vec.shape[0]==num_params
    return vec.reshape(num_spins,num_classes,num_classes)

# Index matrix function. For each sample chain specifies 1D array with indices of present parameters.
# Size: num_conf x num_spins
@jit(nopython=True)
def define_indexmatrix(current_spin,im_type=np.int32):
    index_matrix = np.zeros((num_conf,num_spins), dtype=im_type)
    for k in range(num_conf):            
        ispin = samples_histo[k,1+current_spin]
        for j in range(num_spins):
            jspin = samples_histo[k,1+j]
            index_matrix[k,j] = int(j*num_classes**2 + (ispin-1)*num_classes + jspin-1)
    return index_matrix

# Bound condition function. Returns list of tuples, each specifying bounds for parameter i. 
# Parameter corresponding to (i,j,A,B) should be set to 0 if i==j, A!=B
def bound_condition(i,current_spin):
    if i>=current_spin*num_classes**2 and i<(current_spin+1)*num_classes**2 and ((i-current_spin*num_classes**2)%num_classes)!=(i-current_spin*num_classes**2)//num_classes:
        return (0,0)
    else:
        return (-np.inf,np.inf) 
    
# Find sample weights according to reweighting scheme in PSICOV/CCMPRED

@jit(nopython=True)
def compute_r(samples_histo,const=0.32*0.38):
    avg = 0
    for i in range(num_conf):
        for j in range(i+1,num_conf):
            avg += (samples_histo[i,1:]==samples_histo[j,1:]).sum()
    num_pairs = num_conf*(num_conf-1)//2
    avg = avg/(num_spins*num_pairs)
    r = const/avg
    return r if r<0.5 else 0.5 #as per PSICOV

@jit(nopython=True)
def compute_weights(samples_histo,r,normalize=True):
    sample_weights = np.zeros(num_conf)
    rL = r*num_spins
    for i in range(num_conf):
        for j in range(num_conf):
            if (samples_histo[i,1:]==samples_histo[j,1:]).sum() >= rL:
                sample_weights[i] += 1 
    sample_weights = 1/sample_weights
    if normalize: sample_weights = sample_weights/sample_weights.sum()
    return sample_weights

#r = compute_r(samples_histo)
r = 0.8 #default r for CCMPRED
#sample_weights = compute_weights(samples_histo,r)
sample_weights = np.ones(num_conf)


In [38]:
total_start = time.time()

for current_spin in range(num_spins): 
        
    start = time.time()
    
    print("Reconstructing parameters for spin "+str(current_spin)) 
    
    # Define initial parameter vectors
    x0 = np.zeros(num_params)
    #x0 = np.random.rand(num_params)
    
    # Define weight matrix, i.e presence-absence matrix of size no. parameters x no. configurations
    def define_weightmatrix(current_spin=current_spin):
        weight_matrix = np.zeros((num_conf,num_params))
        for k in range(num_conf):            
            ispin = samples_histo[k,1+current_spin]            
            for j in range(num_spins):
                jspin = samples_histo[k,1+j]
                weight_matrix[k,j*num_classes**2 + (ispin-1)*num_classes + jspin-1] += 1
        return weight_matrix
    
    weight_matrix = define_weightmatrix(current_spin)    

    # Define regularization vectors
    reg_matrix_l1 = lamda_edge*np.ones(num_params)
    #reg_matrix_l1[current_spin*num_classes**2:(current_spin+1)*num_classes**2] = lamda_field 
    reg_matrix_l1[current_spin*num_classes**2:(current_spin+1)*num_classes**2] = 0 
    
    reg_matrix_l2 = np.zeros(num_params)
    reg_matrix_l2[current_spin*num_classes**2:(current_spin+1)*num_classes**2] = lamda_field
    
    # Bounds
    def bound_condition(i):
        if i>=current_spin*num_classes**2 and i<(current_spin+1)*num_classes**2 and ((i-current_spin*num_classes**2)%num_classes)!=(i-current_spin*num_classes**2)//num_classes:
            return (0,0)
        else:
            return (-np.inf,np.inf)
        
    bounds = [bound_condition(i) for i in range(num_params)]
    
    # Smooth approximation of L1 regularization
    alpha = 10000
    
    def smooth_approx(x,alpha=alpha):
        if alpha*x<=0:
            return (2*np.log(1+np.exp(alpha*x)) - alpha*x)/alpha
        else:
            return (2*np.log(1+np.exp(-alpha*x)) + alpha*x)/alpha
    smooth_approx_vec = np.vectorize(smooth_approx)
    
    def smooth_der(x,alpha=alpha):
        if alpha*x<=0:
            return (np.exp(alpha*x)-1)/(1+np.exp(alpha*x))
        else:
            return (1- np.exp(-alpha*x))/(1+np.exp(-alpha*x))
    smooth_der_vec = np.vectorize(smooth_der)
    
    # L1 objective and jacobian   

    def opt_objective_l1(x):

        reg = np.dot(reg_matrix_l1,smooth_approx_vec(x)) 
        reg += np.dot(reg_matrix_l2,x**2) #If L2 regularization on fields
        sample_args = -weight_matrix.dot(x)
        sample_max = np.max(sample_args)
        sample_args = sample_args - sample_max        
        fo_unsummed = (samples_histo[:,0]/num_samples)*np.exp(sample_args)
        return sample_max + np.log(np.sum(fo_unsummed)) + reg
    
    def opt_jacobian_l1(x):
        
        reg_der = np.multiply(reg_matrix_l1,smooth_der_vec(x)) 
        reg_der += np.multiply(reg_matrix_l2,2*x) #if L2 regularization on fields
        
        sample_args = -weight_matrix.dot(x)
        sample_max = np.max(sample_args)
        sample_args = sample_args - sample_max
        
        fo_unsummed = (samples_histo[:,0]/num_samples)*np.exp(sample_args)        
        fo_unsummed = fo_unsummed/np.sum(fo_unsummed)
        
        jac = -weight_matrix.T.dot(fo_unsummed) + reg_der
        
        return jac 
    
 
    sol = minimize(opt_objective_l1, x0,jac=opt_jacobian_l1,bounds=bounds,method="L-BFGS-B",tol=1e-6,options={'ftol':1e-7}) #L-BFGS-B, SLSQP
    
    params = sol['x'].copy()
    params = toND(params)
    
    end = time.time()
    
    print("Time Elapsed for spin ",current_spin," : ", end - start)
    print("Great success!") if sol['success'] else print("Optimization Failed!")
    if not sol['success']: print(sol)
        
    reconstruction[current_spin,:,:,:] = params.copy()
        
total_end = time.time()
print("Total Time for all spins : ", total_end-total_start)


Reconstructing parameters for spin 0
Time Elapsed for spin  0  :  0.17169809341430664
Great success!
Reconstructing parameters for spin 1
Time Elapsed for spin  1  :  0.22779488563537598
Great success!
Reconstructing parameters for spin 2
Time Elapsed for spin  2  :  0.14955997467041016
Great success!
Reconstructing parameters for spin 3
Time Elapsed for spin  3  :  0.17317605018615723
Great success!
Reconstructing parameters for spin 4
Time Elapsed for spin  4  :  0.1874539852142334
Great success!
Reconstructing parameters for spin 5
Time Elapsed for spin  5  :  0.18250107765197754
Great success!
Reconstructing parameters for spin 6
Time Elapsed for spin  6  :  0.21292805671691895
Great success!
Reconstructing parameters for spin 7
Time Elapsed for spin  7  :  0.21927309036254883
Great success!
Total Time for all spins :  1.5296308994293213


In [39]:
trueJ = np.load("gibbscoupling.npy")
print(np.linalg.norm(reconstruction - trueJ))

1438154535750821.8


In [40]:
np.around(trueJ,2)

array([[[[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  , -0.27],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [-0.86,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]]],


       [[[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]]],


       [[[ 0.  ,  0.  ],
         [-0.27,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        [[ 0.  ,  0.  ],
         [ 0.  ,  0.  ]],

        

In [41]:
np.around(reconstruction,2)

array([[[[ 6.22000000e+01,  0.00000000e+00],
         [ 0.00000000e+00, -3.59120000e+02]],

        [[ 2.60514921e+13,  4.71223537e+13],
         [ 3.04256222e+13,  4.35043968e+13]],

        [[ 8.55329894e+13,  5.21511695e+13],
         [ 2.32883334e+13,  4.74169820e+13]],

        [[ 7.95119312e+13,  4.52725829e+13],
         [ 3.22753930e+13,  4.48822942e+13]],

        [[ 8.20466189e+13,  3.05496274e+13],
         [ 4.69983485e+13,  3.66919311e+13]],

        [[ 9.21440738e+13,  3.19717097e+13],
         [ 4.55762663e+13,  7.26586001e+13]],

        [[ 5.42703130e+13,  4.08934808e+13],
         [ 3.68558930e+13,  8.72551216e+13]],

        [[ 4.17822646e+13,  8.31816026e+13],
         [ 0.00000000e+00,  0.00000000e+00]]],


       [[[ 1.95645155e+14,  2.97378626e+14],
         [ 0.00000000e+00,  0.00000000e+00]],

        [[-4.90000000e-01,  0.00000000e+00],
         [ 0.00000000e+00, -2.97000000e+00]],

        [[ 7.91550976e+13,  1.65971567e+14],
         [ 9.90308592e+13,  1.785